<div style="text-align:center; font-size:40px; font-family: 'Times New Roman', Times, serif;font-weight: bold;">
    Tokamak Plasma Disruption Prediction using ML
</div>


<div style ="test-align:left"; font-size:30px; font-family:'Times New Roman'> The data was obtained from the competition published by IAEA AI for Fusion. The following work is not used to participate in the competition; rather, it's more like a self-project performed to understand and learn about the application of AI in the nuclear (fusion) energy sector. More details about the competition can be found at the following link. An F1 score of 0.883116883 was also obtained after submission.  </div>

Link : https://zindi.africa/competitions/multi-machine-disruption-prediction-challenge/submissions

In [56]:
import os
import glob
from tqdm.auto import tqdm


import numpy as np

import utils


from jddb.file_repo import FileRepo

import pandas as pd
from tqdm.auto import tqdm

In [35]:
import zipfile

def extract_zip(zip_files):
    for zip_file, extract_path in zip_files:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_path)


zip_files = [
    ("C-Mod_data.zip", '../tod'),
    ('HL-2A_data.zip', '../tod'),
    ('J-TEXT_data.zip', '../tod'),
    ('../tod/J-TEXT data/data.zip', '../tod/J-TEXT data/')
]


extract_zip(zip_files)


print("Done")

Done


In [36]:
import patoolib


patoolib.extract_archive('../tod/HL-2A data/HL-2A_Data.rar', outdir='../tod/HL-2A data/', program = '/usr/local/bin/unrar')
print("Extraction Successful")

INFO patool: Extracting ../tod/HL-2A data/HL-2A_Data.rar ...
INFO patool: running /usr/local/bin/unrar x -- "/Users/lightvij/Downloads/zindiFusion/tod/HL-2A data/HL-2A_Data.rar"
INFO patool:     with cwd='../tod/HL-2A data/', input=''
INFO patool: ... ../tod/HL-2A data/HL-2A_Data.rar extracted to `../tod/HL-2A data/'.


Extraction Successful


In [3]:

!python basic_data_handling.py
!python basic_processor.py

Traceback (most recent call last):
  File "/Users/lightvij/Downloads/zindiFusion/tod/basic_data_handling.py", line 8, in <module>
    shot = shot_list[20]
           ~~~~~~~~~^^^^
IndexError: list index out of range


In [4]:
directories = {
    "jtext_files": "jtext_train_files",
    "cmod_files": "cmod_train_files",
    "hl_2a_files": "hl2a_train_files",
    "test_files": "cmod_testing_files"
}

for file_name, directory in directories.items():
    path = f"../tod/{directory}"
    os.makedirs(path, exist_ok=True)
    globals()[file_name] = path

print('Done')

Done


In [5]:
import os

folderpath = ["../tod/cmod", "../tod/J-TEXT data", "../tod/HL-2A data"]

def list_subfolders(directory_list):
    for directory in directory_list:
        print(f" Immediate subfolders for: {directory}")
        subfolders = next(os.walk(directory))[1]
        for subfolder in subfolders:
            print(f"  Subfolder: {os.path.join(directory, subfolder)}")
        print("\n")

list_subfolders(folderpath)

 Immediate subfolders for: ../tod/cmod
  Subfolder: ../tod/cmod/CMod_evaluate_20_10_2023
  Subfolder: ../tod/cmod/CMod_train


 Immediate subfolders for: ../tod/J-TEXT data
  Subfolder: ../tod/J-TEXT data/figures
  Subfolder: ../tod/J-TEXT data/processed_data_1k_5k_final


 Immediate subfolders for: ../tod/HL-2A data
  Subfolder: ../tod/HL-2A data/JDDB_repo_2A_5k




In [6]:
directories = ['../tod/cmod/CMod_train/*/*', 
               '../tod/J-TEXT data/processed_data_1k_5k_final/*/*', 
               '../tod/HL-2A data/JDDB_repo_2A_5k/*', 
               '../tod/cmod/CMod_evaluate_20_10_2023/*/*'
              ]
c_mod_paths_files, jtext_paths_files, hl_2a_paths_files, c_mod_test_files = [glob.glob(directory) for directory in directories]

In [7]:

import shutil
from tqdm.notebook import tqdm  # for progress bar

def move_files(source_path, destination_path, file_paths):
    for path in tqdm(file_paths):
        if os.path.exists(path):
            shutil.move(path, f"{destination_path}/{os.path.split(path)[-1]}")
        else:
            print(f"Source file not found: {path}")

# Example usage
move_files(cmod_files, cmod_files, c_mod_paths_files)
move_files(jtext_files, jtext_files, jtext_paths_files)
move_files(hl_2a_files, hl_2a_files, hl_2a_paths_files)


move_files(test_files, test_files, c_mod_test_files)




0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
from jddb.file_repo import FileRepo


def open_repo(path):
    

    file_repo = FileRepo(f"{path}")
    return file_repo

jtext_repo = open_repo(jtext_files)
cmod_repo = open_repo(cmod_files)
hl_2a_repo = open_repo(hl_2a_files)

test_repo = open_repo(test_files)

In [9]:

jtext_shot_list = jtext_repo.get_all_shots()
cmod_shot_list = cmod_repo.get_all_shots()
hl_2a_shot_list = hl_2a_repo.get_all_shots()
test_shot_list = test_repo.get_all_shots()


To return labels for shots in repo specified in the shot_list

In [10]:
def get_labels(repo, shot_list):
    return [repo.read_labels(shot, ['IsDisrupt'])['IsDisrupt'] for shot in shot_list]

In [11]:
def get_maximum_length(tag):
    all_lengths = [len(t) for t in tag]
    return [t for t in tag if len(t) == max(all_lengths)]


In [12]:
def get_tags_and_mini_tags(repo, shots_dict_key):
    tags = [repo.get_tag_list(each_shot) for each_shot in shots_dict_key]
    mini_tags = get_maximum_length(tags)
    tags = np.unique(np.array([cv for te in tags for cv in te]))
    return tags, mini_tags


In [13]:
def new_test_matrix_build(file_shot_repo, shot_list, tags, all_tags):
    """Returns X arrays and tags to drop """
    file_shot_repo_dict = [dict(file_shot_repo.read_data(shot_list[i] , tag_list=tags).items()) for i in range(len(shot_list))]
    tag_to_drop = np.unique(np.array([key for data_dict in file_shot_repo_dict for key in all_tags if key not in data_dict.keys()]))
    file_x = np.asarray([np.nan_to_num(data_dict[f"{key}"]).T for data_dict in file_shot_repo_dict for key in all_tags if key not in tag_to_drop]  , dtype=object).reshape(-1 , len(all_tags) - len(tag_to_drop))
    return file_x , tag_to_drop

def new_train_matrix_build(shot_list, file_shot_repo , tags, all_tags):
    """Returns X & Y arrays and tags_to_drop """
    file_shot_repo_dict = [dict(file_shot_repo.read_data(shot_list[i], tag_list=tags).items()) for i in range(len(shot_list))]
    tag_to_drop = np.unique(np.array([key for data_dict in file_shot_repo_dict for key in all_tags if key not in data_dict.keys()]))
    file_x = np.asarray([np.nan_to_num(data_dict[f"{key}"]).T for data_dict in file_shot_repo_dict for key in all_tags if key not in tag_to_drop], dtype=object).reshape(-1 , len(all_tags) - len(tag_to_drop))

    file_y = np.asarray([np.array(list(file_shot_repo.read_labels(shot_list[i] , ['IsDisrupt']).values())).T.flatten().reshape(-1,) for i in range(len(shot_list))] , dtype=np.int32)
    return file_x , file_y , tag_to_drop

In [14]:
def reform_train_x_sum_of_squares_features(df , mini_tags , shot_list):
    """Returns a reformed dataframe"""
    train_df_dict = {i : [np.square(df[v][i]).sum() for v in range(df.shape[0])] for i in range(df.shape[1])}

    df_final = pd.DataFrame(train_df_dict)
    df_final = df_final.rename(columns  = dict(zip(train_df_dict.keys(), mini_tags )) )
    df_final['Shot_list'] = [f'ID_{shot}' for shot in shot_list]
    return df_final

In [15]:

def rebuild_train_files(train_file_repo , train_shot_list):
    df_final_x = [ ]
    df_final_y = [ ]

    for shot in tqdm(train_shot_list):

        train_tags , mini_tags  = get_tags_and_mini_tags(train_file_repo , [shot])
        train_x_df , train_y_df, _ =  new_train_matrix_build([shot] , train_file_repo , mini_tags[0] , train_tags)
        train_x_df = reform_train_x_sum_of_squares_features(train_x_df , mini_tags[0] , [shot])

        train_y_df = get_labels(train_file_repo , [shot])
        train_y_df = pd.DataFrame(train_y_df)

        df_final_x.append(train_x_df)
        df_final_y.append(train_y_df)

    df_final_x = pd.concat(df_final_x)
    df_final_y = pd.concat(df_final_y)

    df_final_x = df_final_x.reset_index().drop('index' , axis=1)
    df_final_y = df_final_y.reset_index().drop('index' , axis=1)

    return df_final_x , df_final_y

def rebuild_test_files(test_file_repo , test_shot_list):
    df_final_x = [ ]

    for shot in tqdm(test_shot_list):

        test_tags , mini_tags  = get_tags_and_mini_tags(test_file_repo , [shot])
        test_x_df , _ =  new_test_matrix_build(test_file_repo, [shot] , mini_tags[0] , test_tags)
        test_x_df = reform_train_x_sum_of_squares_features(test_x_df , mini_tags[0] , [shot])
        df_final_x.append(test_x_df)

    df_final_x = pd.concat(df_final_x)
    df_final_x = df_final_x.reset_index().drop('index' , axis=1)
    return df_final_x

In [16]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action = 'ignore', category=DeprecationWarning)

train_x_cmod_df , train_y_cmod_df = rebuild_train_files(cmod_repo , cmod_shot_list)
train_x_hl_2a_df , train_y_hl_2a_df = rebuild_train_files(hl_2a_repo , hl_2a_shot_list)
train_x_jtext_df , train_y_jtext_df = rebuild_train_files(jtext_repo , jtext_shot_list)

test_x_cmod_df = rebuild_test_files(test_repo , test_shot_list)


warnings.resetwarnings()
print("--------------------------------  done  ____________________________________________")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/975 [00:00<?, ?it/s]

  0%|          | 0/2136 [00:00<?, ?it/s]

  0%|          | 0/413 [00:00<?, ?it/s]

--------------------------------  done  ____________________________________________


In [17]:
train_x_jtext_df.shape, train_x_hl_2a_df.shape, train_x_cmod_df.shape, test_x_cmod_df.shape


((2136, 90), (975, 86), (20, 173), (413, 198))

In [18]:

test_x_cmod_df.shape

(413, 198)

In [19]:
test_x_cmod_df.head()

,.tci.results:nl_04,Greenwald_fraction,\AXA.chord_1,\AXA.chord_10,\AXA.chord_11,\AXA.chord_12,\AXA.chord_13,\AXA.chord_14,\AXA.chord_15,\AXA.chord_16,...,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_BOT,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_TOP,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP24_GHK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP28_GHK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP19_ABK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP24_ABK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP14_ABK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP11_ABK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP25_GHK,\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP16_GHK
0,7.238038e+42,50.148826,5.912766e+11,5.569227e+13,7.499578e+13,9.531514e+13,1.151346e+14,1.254265e+14,1.319534e+14,1.319970e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.142576e+43,320.385502,1.594344e+12,2.863002e+14,2.880459e+14,2.848301e+14,2.907172e+14,2.840117e+14,2.909774e+14,2.900714e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.505662e+42,83.864994,1.175826e+12,6.103594e+13,7.469048e+13,8.867595e+13,1.088405e+14,1.238158e+14,1.372414e+14,1.410087e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.969224e+41,15.818755,2.814312e+12,8.571182e+13,1.234129e+14,1.667579e+14,2.235311e+14,2.701136e+14,3.086097e+14,3.308350e+14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.905767e+42,80.798441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.178918e+06,1.827706e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
train_x_jtext_df.head(10)

,AXUV_CA_02,AXUV_CA_06,AXUV_CA_10,AXUV_CA_14,AXUV_CB_18,AXUV_CB_22,AXUV_CB_26,AXUV_CB_30,AXUV_CE_66,AXUV_CE_70,...,sxr_cc_054,sxr_cc_055,sxr_cc_056,sxr_cc_057,sxr_cc_058,sxr_cc_059,sxr_cc_060,vl,vs_c3_aa018,Shot_list
0,0.004117,0.000749,0.001913,0.002700,274.472462,295.295605,603.843927,6.471056,166.282142,205.286680,...,452.313538,350.903274,244.752777,156.079370,93.583202,51.789313,26.497771,1443.521017,502.279370,ID_1052049
1,151.223528,187.277583,356.875999,26.823558,122.832797,137.164923,199.067237,3.552601,73.866313,76.732613,...,123.086532,77.315146,44.482799,25.275620,14.294747,7.872120,4.296249,647.892360,80.525575,ID_1055276
2,414.949817,582.335424,1045.279103,94.264041,136.709745,159.486597,110.270022,0.997493,236.431201,292.150984,...,549.197635,414.561723,276.055031,165.013037,90.675153,44.432935,20.986154,1425.770464,610.909144,ID_1058014
3,163.595429,179.596307,249.261059,35.856845,121.040730,133.825573,141.431449,4.014172,73.026711,83.947422,...,68.014465,43.486788,24.191642,13.330661,7.190716,3.864045,1.736649,1153.602130,86.560705,ID_1053208
4,350.828240,0.003829,0.001443,222.140584,0.002894,0.002414,0.002436,0.003031,16.722326,16.868234,...,208.509061,151.535359,95.486642,56.170084,31.308580,16.315708,7.770002,1401.105023,443.656876,ID_1054172
5,95.067434,100.448165,224.313532,25.456871,73.936166,67.236181,114.377625,2.636565,43.790068,47.799708,...,33.044671,24.080121,15.820584,9.351247,4.989020,2.601083,1.258517,1731.614563,73.022255,ID_1055333
6,79.037341,115.424285,674.005582,50.687582,55.471493,58.172208,210.735332,1.537289,30.477127,35.427831,...,63.377469,46.297928,29.377785,17.186371,9.461691,4.946630,2.405480,928.618417,71.518699,ID_1054871
7,174.782237,201.855434,507.139855,86.939932,147.833099,126.375550,174.916560,1.679552,88.480638,92.271925,...,85.397230,55.620815,31.170364,16.786896,8.730465,4.309283,2.016777,1393.940092,259.027230,ID_1055763
8,43.389604,49.690541,169.345336,19.015541,32.687642,28.439793,65.748719,1.028850,19.385528,21.376731,...,30.036976,18.858453,11.238565,6.679940,3.894762,2.268096,1.088235,498.197456,11.966925,ID_1053064
9,444.837614,626.958838,1217.848857,116.083587,226.687543,245.822944,172.302511,2.487181,248.440142,311.767255,...,440.785988,334.663310,222.862227,131.908943,73.684662,37.028800,18.008817,1850.328264,1046.354808,ID_1057477


In [21]:
train_x_hl_2a_df.head(2)

,CCO-DF:DENSITY1,CCO-DF:GASFBOUT,CCO-LFB:LFBBOH,CCO-LFB:LFBBT,CCO-LFB:LFBBV,CCO-LFB:LFBIRF,CCO-LFB:LFBMP1,CCO-LFB:LFBMP2,CCO-LFB:LFDH,CCO-LFB:LFDV,...,DS-SXR-SXA:SX18,DS-SXR-SXA:SX19,DS-SXR-SXA:SX20,DS-TMP:PUFFCTRL,EFIT_q95,GW_Ratio,Ip_Error,RadiatedFraction,RotatingModeProxy,Shot_list
0,6288.000993,262.509102,77430.602017,957218.279478,297574.373035,844.555869,358188.949523,500183.573067,1445.806329,4732.936257,...,808.132662,147.985045,35.787802,368132.144570,22492.701167,484.400487,30048.563997,2.009755e+04,0.015177,ID_36694
1,1550.721931,241.610116,45348.930443,555196.410884,104686.312142,363.024795,0.101659,0.266430,569.987963,3887.157997,...,0.862126,2.328947,1.489772,515586.683379,15670.171046,105.818840,11272.650498,1.404387e+09,0.082587,ID_36047


In [22]:
cmod_train_features = train_x_cmod_df.columns.tolist()
print(cmod_train_features)
n = len(cmod_train_features)
print(n)
train_x_cmod_df.shape

['.tci.results:nl_04', 'Greenwald_fraction', '\\AXA.chord_1', '\\AXA.chord_10', '\\AXA.chord_11', '\\AXA.chord_12', '\\AXA.chord_13', '\\AXA.chord_14', '\\AXA.chord_15', '\\AXA.chord_16', '\\AXA.chord_17', '\\AXA.chord_18', '\\AXA.chord_19', '\\AXA.chord_2', '\\AXA.chord_20', '\\AXA.chord_21', '\\AXA.chord_22', '\\AXA.chord_3', '\\AXA.chord_4', '\\AXA.chord_5', '\\AXA.chord_6', '\\AXA.chord_7', '\\AXA.chord_8', '\\AXA.chord_9', '\\AXJ.chord_1', '\\AXJ.chord_10', '\\AXJ.chord_11', '\\AXJ.chord_12', '\\AXJ.chord_13', '\\AXJ.chord_14', '\\AXJ.chord_15', '\\AXJ.chord_16', '\\AXJ.chord_17', '\\AXJ.chord_18', '\\AXJ.chord_19', '\\AXJ.chord_2', '\\AXJ.chord_20', '\\AXJ.chord_21', '\\AXJ.chord_22', '\\AXJ.chord_23', '\\AXJ.chord_24', '\\AXJ.chord_25', '\\AXJ.chord_26', '\\AXJ.chord_27', '\\AXJ.chord_3', '\\AXJ.chord_4', '\\AXJ.chord_5', '\\AXJ.chord_6', '\\AXJ.chord_7', '\\AXJ.chord_8', '\\AXJ.chord_9', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP01_GHK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_G

(20, 173)

In [40]:
col_feat_b = test_x_cmod_df.columns.tolist()     #column names before renaming
print(col_feat_b)


['plasma_current', 'toroidal_magnetic_field', 'horizontal_displacement', 'vertical_displacement', 'line_integral_density(center_chord)', 'c3_radiation', 'loop_voltage', 'axuv_01', 'axuv_02', 'axuv_03', 'axuv_04', 'axuv_05', 'axuv_06', 'axuv_07', 'axuv_08', 'axuv_09', 'axuv_10', 'axuv_11', 'axuv_12', 'axuv_13', 'axuv_14', 'axuv_15', 'axuv_16', 'axuv_17', 'axuv_18', 'axuv_19', 'axuv_20', 'axuv_21', 'axuv_22', 'soft_x_ray_01', 'soft_x_ray_02', 'soft_x_ray_03', 'soft_x_ray_04', 'soft_x_ray_05', 'soft_x_ray_06', 'soft_x_ray_07', 'soft_x_ray_08', 'soft_x_ray_09', 'soft_x_ray_10', 'soft_x_ray_11', 'soft_x_ray_12', 'soft_x_ray_13', 'soft_x_ray_14', 'soft_x_ray_15', 'soft_x_ray_16', 'soft_x_ray_17', 'soft_x_ray_18', 'soft_x_ray_19', 'soft_x_ray_20', 'soft_x_ray_21', 'soft_x_ray_22', 'soft_x_ray_23', 'soft_x_ray_24', 'soft_x_ray_25', 'soft_x_ray_26', 'soft_x_ray_27', 'soft_x_ray_28', 'soft_x_ray_29', 'soft_x_ray_30', 'soft_x_ray_31', 'soft_x_ray_32', 'soft_x_ray_33', 'soft_x_ray_34', 'soft_x_ray

<div style="font-size: 30px;">
    Feature Engineering</div>

<div style="font-size: 20px;">Renaming and Processing CMOD files</div>


In [28]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,AXUV_17,AXUV_18,AXUV_19,AXUV_20,AXUV_21,AXUV_22,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,soft-X-ray_21,soft-X-ray_22,soft-X-ray_23,soft-X-ray_24,soft-X-ray_25,soft-X-ray_26,soft-X-ray_27,soft-X-ray_28,soft-X-ray_29,soft-X-ray_30,soft-X-ray_31,soft-X-ray_32,soft-X-ray_33,soft-X-ray_34,soft-X-ray_35,soft-X-ray_36,soft-X-ray_37,soft-X-ray_38,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,poloidal Mirnov probes_13,poloidal Mirnov probes_14,poloidal Mirnov probes_15,poloidal Mirnov probes_16,poloidal Mirnov probes_17,poloidal Mirnov probes_18,toroidal Mirnov probes_01,toroidal Mirnov probes_02,toroidal Mirnov probes_03,toroidal Mirnov probes_04,toroidal Mirnov probes_05,toroidal Mirnov probes_06,toroidal Mirnov probes_07,toroidal Mirnov probes_09,toroidal Mirnov probes_10,Density_Measurement1,Density_Measurement2,Density_Measurement3,Density_Measurement4,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy,unknown_BP_KA_TOP,unknown_BP_EF_TOP,unknown_BP_KA_BOT,unknown_aminor,unknown_betan,unknown_kappa,unknown_li,unknown_wplasm".replace(' ','_').replace('-', '_').lower().split(',')
cmod_feats =["\\ip", "\\btor", '\\efit_aeqdsk:rmagx', '\\efit_aeqdsk:zmagx', '.tci.results:nl_04','\\twopi_diode', '\\efit_aeqdsk:vloopt','\\AXA.chord_1 & \\AXJ.chord_1', '\\AXA.chord_2 & \\AXJ.chord_2', '\\AXA.chord_3 & \\AXJ.chord_3', '\\AXA.chord_4 & \\AXJ.chord_4', '\\AXA.chord_5 & \\AXJ.chord_5', '\\AXA.chord_6 & \\AXJ.chord_6', '\\AXA.chord_7 & \\AXJ.chord_7', '\\AXA.chord_8 & \\AXJ.chord_8', '\\AXA.chord_9 & \\AXJ.chord_9', '\\AXA.chord_10 & \\AXJ.chord_10', '\\AXA.chord_11 & \\AXJ.chord_11', '\\AXA.chord_12 & \\AXJ.chord_12', '\\AXA.chord_13 & \\AXJ.chord_13', '\\AXA.chord_14 & \\AXJ.chord_14', '\\AXA.chord_15 & \\AXJ.chord_15', '\\AXA.chord_16 & \\AXJ.chord_16', '\\AXA.chord_17 & \\AXJ.chord_17', '\\AXA.chord_18 & \\AXJ.chord_18', '\\AXA.chord_19 & \\AXJ.chord_19', '\\AXA.chord_20 & \\AXJ.chord_20', '\\AXA.chord_21 & \\AXJ.chord_21', '\\AXA.chord_22 & \\AXJ.chord_22','\\xtomo::top.brightnesses.array_1:chord_01 & \\xtomo::top.brightnesses.array_3:chord_01', '\\xtomo::top.brightnesses.array_1:chord_02 & \\xtomo::top.brightnesses.array_3:chord_02', '\\xtomo::top.brightnesses.array_1:chord_03 & \\xtomo::top.brightnesses.array_3:chord_03', '\\xtomo::top.brightnesses.array_1:chord_04 & \\xtomo::top.brightnesses.array_3:chord_04', '\\xtomo::top.brightnesses.array_1:chord_05 & \\xtomo::top.brightnesses.array_3:chord_05', '\\xtomo::top.brightnesses.array_1:chord_06 & \\xtomo::top.brightnesses.array_3:chord_06', '\\xtomo::top.brightnesses.array_1:chord_07 & \\xtomo::top.brightnesses.array_3:chord_07', '\\xtomo::top.brightnesses.array_1:chord_08 & \\xtomo::top.brightnesses.array_3:chord_08', '\\xtomo::top.brightnesses.array_1:chord_09 & \\xtomo::top.brightnesses.array_3:chord_09', '\\xtomo::top.brightnesses.array_1:chord_10 & \\xtomo::top.brightnesses.array_3:chord_10', '\\xtomo::top.brightnesses.array_1:chord_11 & \\xtomo::top.brightnesses.array_3:chord_11', '\\xtomo::top.brightnesses.array_1:chord_12 & \\xtomo::top.brightnesses.array_3:chord_12', '\\xtomo::top.brightnesses.array_1:chord_13 & \\xtomo::top.brightnesses.array_3:chord_13', '\\xtomo::top.brightnesses.array_1:chord_14 & \\xtomo::top.brightnesses.array_3:chord_14', '\\xtomo::top.brightnesses.array_1:chord_15 & \\xtomo::top.brightnesses.array_3:chord_15', '\\xtomo::top.brightnesses.array_1:chord_16 & \\xtomo::top.brightnesses.array_3:chord_16', '\\xtomo::top.brightnesses.array_1:chord_17 & \\xtomo::top.brightnesses.array_3:chord_17', '\\xtomo::top.brightnesses.array_1:chord_18 & \\xtomo::top.brightnesses.array_3:chord_18', '\\xtomo::top.brightnesses.array_1:chord_19 & \\xtomo::top.brightnesses.array_3:chord_19', '\\xtomo::top.brightnesses.array_1:chord_20 & \\xtomo::top.brightnesses.array_3:chord_20', '\\xtomo::top.brightnesses.array_1:chord_21 & \\xtomo::top.brightnesses.array_3:chord_21', '\\xtomo::top.brightnesses.array_1:chord_22 & \\xtomo::top.brightnesses.array_3:chord_22', '\\xtomo::top.brightnesses.array_1:chord_23 & \\xtomo::top.brightnesses.array_3:chord_23', '\\xtomo::top.brightnesses.array_1:chord_24 & \\xtomo::top.brightnesses.array_3:chord_24', '\\xtomo::top.brightnesses.array_1:chord_25 & \\xtomo::top.brightnesses.array_3:chord_25','\\xtomo::top.brightnesses.array_1:chord_26 & \\xtomo::top.brightnesses.array_3:chord_26', '\\xtomo::top.brightnesses.array_1:chord_27 & \\xtomo::top.brightnesses.array_3:chord_27', '\\xtomo::top.brightnesses.array_1:chord_28 & \\xtomo::top.brightnesses.array_3:chord_28', '\\xtomo::top.brightnesses.array_1:chord_29 & \\xtomo::top.brightnesses.array_3:chord_29', '\\xtomo::top.brightnesses.array_1:chord_30 & \\xtomo::top.brightnesses.array_3:chord_30', '\\xtomo::top.brightnesses.array_1:chord_31 & \\xtomo::top.brightnesses.array_3:chord_31', '\\xtomo::top.brightnesses.array_1:chord_32 & \\xtomo::top.brightnesses.array_3:chord_32', '\\xtomo::top.brightnesses.array_1:chord_33 & \\xtomo::top.brightnesses.array_3:chord_33', '\\xtomo::top.brightnesses.array_1:chord_34 & \\xtomo::top.brightnesses.array_3:chord_34', '\\xtomo::top.brightnesses.array_1:chord_35 & \\xtomo::top.brightnesses.array_3:chord_35', '\\xtomo::top.brightnesses.array_1:chord_36 & \\xtomo::top.brightnesses.array_3:chord_36', '\\xtomo::top.brightnesses.array_1:chord_37 & \\xtomo::top.brightnesses.array_3:chord_37', '\\xtomo::top.brightnesses.array_1:chord_38 & \\xtomo::top.brightnesses.array_3:chord_38', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP01_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP03_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP07_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP07_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP08_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP10_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP11_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP12_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP13_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP14_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP08_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP09_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP1T_GHK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP20_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP2T_GHK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP3T_GHK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP4T_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP5T_ABK', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP6T_ABK','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_AB_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_BC_BOT', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_BC_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_EF_BOT','Greenwald_fraction','n1_norm','\\efit_aeqdsk:q95','ip_err_frac',  'radiated_frac', 'rotating_mode' , '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_TOP','\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_EF_TOP', '\\MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP_KA_BOT','\\efit_aeqdsk:aminor', '\\efit_aeqdsk:betan', '\\efit_aeqdsk:kappa', '\\efit_aeqdsk:li', '\\efit_aeqdsk:wplasm']
shot_ids = [i for i in test_x_cmod_df['Shot_list'].values]

c = "\\"
for i in range(22):
    test_x_cmod_df[f'{c}AXA.chord_{i+1} & {c}AXJ.chord_{i+1}'] = test_x_cmod_df[f'{c}AXA.chord_{i+1}'] / test_x_cmod_df[f'{c}AXJ.chord_{i+1}']
    train_x_cmod_df[f'{c}AXA.chord_{i+1} & {c}AXJ.chord_{i+1}'] = train_x_cmod_df[f'{c}AXA.chord_{i+1}'] / train_x_cmod_df[f'{c}AXJ.chord_{i+1}']

for i in range(1, 39):
    if i < 10:
        test_x_cmod_df[f"{c}xtomo::top.brightnesses.array_1:chord_0{i} & {c}xtomo::top.brightnesses.array_3:chord_0{i}"] =  test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i}'] / test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_0{i}']
        train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i} & {c}xtomo::top.brightnesses.array_3:chord_0{i}'] = train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_0{i}'] / train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_0{i}']
    else:
        test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i} & {c}xtomo::top.brightnesses.array_3:chord_{i}'] = test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i}'] / test_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_{i}']
        train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i} & {c}xtomo::top.brightnesses.array_3:chord_{i}'] = train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_1:chord_{i}'] / train_x_cmod_df[f'{c}xtomo::top.brightnesses.array_3:chord_{i}']

train_x_cmod_df[[f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP01_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP02_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP03_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP04_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP05_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP06_ABK', f'{c}MAGNETICS::TOP.ACTIVE_MHD.SIGNALS:BP09_ABK']] = np.NaN



train_x_cmod_df = train_x_cmod_df[cmod_feats].rename(columns = dict(zip(cmod_feats , Diagnostics)) )

test_x_cmod_df = test_x_cmod_df[cmod_feats].rename(columns = dict(zip(cmod_feats , Diagnostics)) )

train_y_cmod_df  = train_y_cmod_df.rename(columns={0:'Is_disrupt'})
train_y_cmod_df = train_y_cmod_df.reset_index().drop('index' , axis=1)

train_df = train_x_cmod_df
train_df['Is_disrupt'] = train_y_cmod_df['Is_disrupt'].astype(int)

test_df = test_x_cmod_df
test_df['Shot_list'] = shot_ids 


In [29]:
train_df.shape, test_df.shape

((20, 113), (413, 113))

In [37]:
col_feats_a = test_df.columns.tolist()
print(col_feats_a)
print('Number of columns after processing = ', len(col_feats_a))
#print(col_feat_b)
print('number of columns before processing = ', len(col_feat_b))

['plasma_current', 'toroidal_magnetic_field', 'horizontal_displacement', 'vertical_displacement', 'line_integral_density(center_chord)', 'c3_radiation', 'loop_voltage', 'axuv_01', 'axuv_02', 'axuv_03', 'axuv_04', 'axuv_05', 'axuv_06', 'axuv_07', 'axuv_08', 'axuv_09', 'axuv_10', 'axuv_11', 'axuv_12', 'axuv_13', 'axuv_14', 'axuv_15', 'axuv_16', 'axuv_17', 'axuv_18', 'axuv_19', 'axuv_20', 'axuv_21', 'axuv_22', 'soft_x_ray_01', 'soft_x_ray_02', 'soft_x_ray_03', 'soft_x_ray_04', 'soft_x_ray_05', 'soft_x_ray_06', 'soft_x_ray_07', 'soft_x_ray_08', 'soft_x_ray_09', 'soft_x_ray_10', 'soft_x_ray_11', 'soft_x_ray_12', 'soft_x_ray_13', 'soft_x_ray_14', 'soft_x_ray_15', 'soft_x_ray_16', 'soft_x_ray_17', 'soft_x_ray_18', 'soft_x_ray_19', 'soft_x_ray_20', 'soft_x_ray_21', 'soft_x_ray_22', 'soft_x_ray_23', 'soft_x_ray_24', 'soft_x_ray_25', 'soft_x_ray_26', 'soft_x_ray_27', 'soft_x_ray_28', 'soft_x_ray_29', 'soft_x_ray_30', 'soft_x_ray_31', 'soft_x_ray_32', 'soft_x_ray_33', 'soft_x_ray_34', 'soft_x_ray

In [41]:
train_df.to_csv('../tod/processed_data/train_cmod.csv' , index=False) 
test_df.to_csv('../tod/processed_data/test_cmod.csv' , index=True)   

/Users/lightvij/anaconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


<div style="font-size: 20px;">Renaming and Processing J-TEXT files</div>

In [42]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,AXUV_17,AXUV_18,AXUV_19,AXUV_20,AXUV_21,AXUV_22,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,soft-X-ray_21,soft-X-ray_22,soft-X-ray_23,soft-X-ray_24,soft-X-ray_25,soft-X-ray_26,soft-X-ray_27,soft-X-ray_28,soft-X-ray_29,soft-X-ray_30,soft-X-ray_31,soft-X-ray_32,soft-X-ray_33,soft-X-ray_34,soft-X-ray_35,soft-X-ray_36,soft-X-ray_37,soft-X-ray_38,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,toroidal Mirnov probes_01,toroidal Mirnov probes_02,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy".replace(' ','_').replace('-', '_').lower().split(',')
jtext_feats = "ip,bt,dx,dy,polaris_den_v09,vs_c3_aa018,vl,AXUV_CA_02,AXUV_CA_06,AXUV_CA_10,AXUV_CA_14,AXUV_CB_18,AXUV_CB_22,AXUV_CB_26,AXUV_CB_30,AXUV_CE_66,AXUV_CE_70,AXUV_CE_74,AXUV_CE_78,AXUV_CF_82,AXUV_CF_86,AXUV_CF_90,AXUV_CF_94,AXUV_CF_95,AXUV_CF_96,AXUV_CF_97,AXUV_CF_98,AXUV_CF_99,AXUV_CF_100,sxr_cb_020,sxr_cb_021,sxr_cb_022,sxr_cb_023,sxr_cb_024,sxr_cb_025,sxr_cb_026,sxr_cb_027,sxr_cb_028,sxr_cb_032,sxr_cb_036,sxr_cb_037,sxr_cb_038,sxr_cb_039,sxr_cb_040,sxr_cb_041,sxr_cb_042,sxr_cb_043,sxr_cb_044,sxr_cc_036,sxr_cc_037,sxr_cc_038,sxr_cc_039,sxr_cc_040,sxr_cc_041,sxr_cc_042,sxr_cc_043,sxr_cc_044,sxr_cc_052,sxr_cc_053,sxr_cc_054,sxr_cc_055,sxr_cc_056,sxr_cc_057,sxr_cc_058,sxr_cc_059,sxr_cc_060,sxr_cc_061,MA_POL_CA01T,MA_POL_CA03T,MA_POL_CA05T,MA_POL_CA07T,MA_POL_CA09T,MA_POL_CA11T,MA_POL_CA13T,MA_POL_CA15T,MA_POL_CA17T,MA_POL_CA19T,MA_POL_CA21T,MA_POL_CA23T,MA_TOR1_R01,MA_TOR1_R02,ne_nG,n = 1 amplitude,qa_proxy,ip_error,radiation_proxy,rotating_mode_proxy".split(',')


train_x_jtext_df[['AXUV_CF_95', 'AXUV_CF_96', 'AXUV_CF_97', 'AXUV_CF_98', 'AXUV_CF_99', 'AXUV_CF_100', 'sxr_cc_061', 'n = 1 amplitude']]=0
train_x_jtext_df = train_x_jtext_df[jtext_feats].rename(columns = dict(zip(jtext_feats , Diagnostics)) )

train_y_jtext_df  = train_y_jtext_df.rename(columns={0:'Is_disrupt'})
train_y_jtext_df = train_y_jtext_df.reset_index().drop('index' , axis=1)

train_df_ = train_x_jtext_df
train_df_['Is_disrupt'] = train_y_jtext_df['Is_disrupt'].astype(int)

train_df_.to_csv('../tod/processed_data/train_jtext.csv' , index=False)  

<div style="font-size: 20px;">Renaming and Processing HL-2A files</div>

In [43]:
Diagnostics = "plasma current,toroidal magnetic field,horizontal displacement,vertical displacement,line integral density(center chord),C3 radiation,loop voltage,AXUV_01,AXUV_02,AXUV_03,AXUV_04,AXUV_05,AXUV_06,AXUV_07,AXUV_08,AXUV_09,AXUV_10,AXUV_11,AXUV_12,AXUV_13,AXUV_14,AXUV_15,AXUV_16,soft-X-ray_01,soft-X-ray_02,soft-X-ray_03,soft-X-ray_04,soft-X-ray_05,soft-X-ray_06,soft-X-ray_07,soft-X-ray_08,soft-X-ray_09,soft-X-ray_10,soft-X-ray_11,soft-X-ray_12,soft-X-ray_13,soft-X-ray_14,soft-X-ray_15,soft-X-ray_16,soft-X-ray_17,soft-X-ray_18,soft-X-ray_19,soft-X-ray_20,poloidal Mirnov probes_01,poloidal Mirnov probes_02,poloidal Mirnov probes_03,poloidal Mirnov probes_04,poloidal Mirnov probes_05,poloidal Mirnov probes_06,poloidal Mirnov probes_07,poloidal Mirnov probes_08,poloidal Mirnov probes_09,poloidal Mirnov probes_10,poloidal Mirnov probes_11,poloidal Mirnov probes_12,poloidal Mirnov probes_13,poloidal Mirnov probes_14,poloidal Mirnov probes_15,poloidal Mirnov probes_16,poloidal Mirnov probes_17,poloidal Mirnov probes_18,toroidal Mirnov probes_01,toroidal Mirnov probes_02,toroidal Mirnov probes_03,toroidal Mirnov probes_04,toroidal Mirnov probes_05,toroidal Mirnov probes_06,toroidal Mirnov probes_07,toroidal Mirnov probes_09,toroidal Mirnov probes_10,Density_Measurement1,Density_Measurement2,Density_Measurement3,Density_Measurement4,Greenwald fraction,LM proxy,q95 proxy,IP error fraction,Radiation fraction,Rotating mode proxy,unknown_LFBBOH,unknown_PUFFCTRL,unknown_LFBBV,unknown_LFBIRF,unknown_LFBMP1,unknown_LFBMP2".replace(' ','_').replace('-', '_').lower().split(',')
hl_2a_feats = "CCO-LFB:LFEX-IP,CCO-LFB:LFBBT,CCO-LFB:LFDH,CCO-LFB:LFDV,CCO-DF:DENSITY1,CCO-DF:GASFBOUT,DS-EMD-ROG:VL-FILTER,DS-BM-AB:BOLD01,DS-BM-AB:BOLD02,DS-BM-AB:BOLD03,DS-BM-AB:BOLD04,DS-BM-AB:BOLD05,DS-BM-AB:BOLD06,DS-BM-AB:BOLD07,DS-BM-AB:BOLD08,DS-BM-AB:BOLD09,DS-BM-AB:BOLD10,DS-BM-AB:BOLD11,DS-BM-AB:BOLD12,DS-BM-AB:BOLD13,DS-BM-AB:BOLD14,DS-BM-AB:BOLD15,DS-BM-AB:BOLD16,DS-SXR-SXA:SX01,DS-SXR-SXA:SX02,DS-SXR-SXA:SX03,DS-SXR-SXA:SX04,DS-SXR-SXA:SX05,DS-SXR-SXA:SX06,DS-SXR-SXA:SX07,DS-SXR-SXA:SX08,DS-SXR-SXA:SX09,DS-SXR-SXA:SX10,DS-SXR-SXA:SX11,DS-SXR-SXA:SX12,DS-SXR-SXA:SX13,DS-SXR-SXA:SX14,DS-SXR-SXA:SX15,DS-SXR-SXA:SX16,DS-SXR-SXA:SX17,DS-SXR-SXA:SX18,DS-SXR-SXA:SX19,DS-SXR-SXA:SX20,DS-EMD-MP:MPOL-01,DS-EMD-MP:MPOL-02,DS-EMD-MP:MPOL-03,DS-EMD-MP:MPOL-04,DS-EMD-MP:MPOL-05,DS-EMD-MP:MPOL-06,DS-EMD-MP:MPOL-07,DS-EMD-MP:MPOL-08,DS-EMD-MP:MPOL-09,DS-EMD-MP:MPOL-10,DS-EMD-MP:MPOL-11,DS-EMD-MP:MPOL-12,DS-EMD-MP:MPOL-13,DS-EMD-MP:MPOL-14,DS-EMD-MP:MPOL-15,DS-EMD-MP:MPOL-16,DS-EMD-MP:MPOL-17,DS-EMD-MP:MPOL-18,DS-EMD-MP:NPOL-01,DS-EMD-MP:NPOL-02,DS-EMD-MP:NPOL-03,DS-EMD-MP:NPOL-04,DS-EMD-MP:NPOL-05,DS-EMD-MP:NPOL-06,DS-EMD-MP:NPOL-07,DS-EMD-MP:NPOL-09,DS-EMD-MP:NPOL-10,DS-FIR:FIR-SPH1A,DS-FIR:FIR-SPH2A,DS-FIR:FIR-SPH3A,DS-FIR:FIR-SPH4A,GW_Ratio,n1_norm,EFIT_q95,Ip_Error,RadiatedFraction,RotatingModeProxy,CCO-LFB:LFBBOH,DS-TMP:PUFFCTRL,CCO-LFB:LFBBV,CCO-LFB:LFBIRF,CCO-LFB:LFBMP1,CCO-LFB:LFBMP2".split(",")

train_x_hl_2a_df['n1_norm'] = 0
train_x_hl_2a_df = train_x_hl_2a_df[hl_2a_feats].rename(columns = dict(zip(hl_2a_feats , Diagnostics)) )

train_y_hl_2a_df  = train_y_hl_2a_df.rename(columns={0:'Is_disrupt'})
train_y_hl_2a_df = train_y_hl_2a_df.reset_index().drop('index' , axis=1)

train_df_ = train_x_hl_2a_df
train_df_['Is_disrupt'] = train_y_hl_2a_df['Is_disrupt'].astype(int)

train_df_.to_csv('../tod/processed_data/train_hl2a.csv' , index=False)  #2404 observations

/Users/lightvij/anaconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


<div style="font-size: 20px;">Further Processing</div>

In [44]:

import pandas as pd
train_cmod_df = pd.read_csv('../tod/processed_data/train_cmod.csv')
train_jtext_df = pd.read_csv('../tod/processed_data/train_jtext.csv')
train_hl_2a_df = pd.read_csv('../tod/processed_data/train_hl2a.csv')

test_cmod_df = pd.read_csv('../tod/processed_data/test_cmod.csv')


In [45]:
train_df = pd.concat([train_cmod_df , train_jtext_df , train_hl_2a_df])

In [46]:
train_df.to_excel('../tod/processed_data/train.xlsx', index=False)
train_df.to_csv('../tod/processed_data/train.xlsx', index=False)


/Users/lightvij/anaconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:2538: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [47]:
train_df.shape, test_cmod_df.shape

((3131, 119), (413, 114))

In [48]:
train_df['programmed_plasma_current'] = train_df['plasma_current'] - train_df['ip_error_fraction']
test_cmod_df['programmed_plasma_current'] = test_cmod_df['plasma_current'] - test_cmod_df['ip_error_fraction']

In [53]:
train_df.head(5)

,plasma_current,toroidal_magnetic_field,horizontal_displacement,vertical_displacement,line_integral_density(center_chord),c3_radiation,loop_voltage,axuv_01,axuv_02,axuv_03,...,unknown_li,unknown_wplasm,Is_disrupt,unknown_lfbboh,unknown_puffctrl,unknown_lfbbv,unknown_lfbirf,unknown_lfbmp1,unknown_lfbmp2,programmed_plasma_current
0,2.107438e+15,302640.34,5.327099e+06,1073.739047,7.390368e+42,312965660.0,5561.634209,inf,inf,0.004912,...,2267.725996,2.611100e+12,1,NaN,NaN,NaN,NaN,NaN,NaN,2.107438e+15
1,4.095709e+14,280047.78,4.023334e+06,31.727082,1.013297e+42,33170362.0,2929.295372,inf,inf,0.000953,...,2329.930054,8.617792e+10,1,NaN,NaN,NaN,NaN,NaN,NaN,4.095709e+14
2,8.966099e+14,290311.06,4.637451e+06,195.638329,4.358243e+42,351733630.0,4056.871891,inf,inf,0.003227,...,2380.996249,3.078669e+11,1,NaN,NaN,NaN,NaN,NaN,NaN,8.966099e+14
3,1.348079e+15,282419.22,8.514665e+06,1332.202990,2.588666e+43,243026460.0,3399.162007,inf,inf,0.001282,...,4375.244812,6.877778e+12,0,NaN,NaN,NaN,NaN,NaN,NaN,1.348079e+15
4,1.338732e+15,306852.70,8.500387e+06,932.798940,2.946724e+42,48145970.0,3960.202012,inf,inf,0.000622,...,5352.263695,5.081751e+11,0,NaN,NaN,NaN,NaN,NaN,NaN,1.338732e+15


In [50]:
column_names = test_cmod_df.columns.tolist()
print("Column names:", column_names)
print('Number of Cols in test file = ', len(column_names))

Column names: ['Unnamed: 0', 'plasma_current', 'toroidal_magnetic_field', 'horizontal_displacement', 'vertical_displacement', 'line_integral_density(center_chord)', 'c3_radiation', 'loop_voltage', 'axuv_01', 'axuv_02', 'axuv_03', 'axuv_04', 'axuv_05', 'axuv_06', 'axuv_07', 'axuv_08', 'axuv_09', 'axuv_10', 'axuv_11', 'axuv_12', 'axuv_13', 'axuv_14', 'axuv_15', 'axuv_16', 'axuv_17', 'axuv_18', 'axuv_19', 'axuv_20', 'axuv_21', 'axuv_22', 'soft_x_ray_01', 'soft_x_ray_02', 'soft_x_ray_03', 'soft_x_ray_04', 'soft_x_ray_05', 'soft_x_ray_06', 'soft_x_ray_07', 'soft_x_ray_08', 'soft_x_ray_09', 'soft_x_ray_10', 'soft_x_ray_11', 'soft_x_ray_12', 'soft_x_ray_13', 'soft_x_ray_14', 'soft_x_ray_15', 'soft_x_ray_16', 'soft_x_ray_17', 'soft_x_ray_18', 'soft_x_ray_19', 'soft_x_ray_20', 'soft_x_ray_21', 'soft_x_ray_22', 'soft_x_ray_23', 'soft_x_ray_24', 'soft_x_ray_25', 'soft_x_ray_26', 'soft_x_ray_27', 'soft_x_ray_28', 'soft_x_ray_29', 'soft_x_ray_30', 'soft_x_ray_31', 'soft_x_ray_32', 'soft_x_ray_33', 

In [54]:
col_features = ['plasma_current', 'toroidal_magnetic_field', 'horizontal_displacement', 'vertical_displacement', 'line_integral_density(center_chord)', 'c3_radiation',
 'loop_voltage', 'axuv_01', 'axuv_02', 'axuv_03', 'axuv_04', 'axuv_05', 'axuv_06', 'axuv_07', 'axuv_08', 'axuv_09', 'axuv_10', 'axuv_11', 'axuv_12', 'axuv_13',
 'axuv_14', 'axuv_15', 'axuv_16', 'axuv_17', 'axuv_18', 'axuv_19', 'axuv_20', 'axuv_21', 'axuv_22', 'soft_x_ray_01',
 'soft_x_ray_02', 'soft_x_ray_03', 'soft_x_ray_04', 'soft_x_ray_05', 'soft_x_ray_06', 'soft_x_ray_07', 'soft_x_ray_08', 'soft_x_ray_09',
 'soft_x_ray_10', 'soft_x_ray_11', 'soft_x_ray_12', 'soft_x_ray_13', 'soft_x_ray_14', 'soft_x_ray_15', 'soft_x_ray_16', 'soft_x_ray_17',
 'soft_x_ray_18', 'soft_x_ray_19', 'soft_x_ray_20', 'soft_x_ray_21', 'soft_x_ray_22', 'soft_x_ray_23', 'soft_x_ray_24', 'soft_x_ray_25',
 'soft_x_ray_26', 'soft_x_ray_27', 'soft_x_ray_28', 'soft_x_ray_29', 'soft_x_ray_30', 'soft_x_ray_31', 'soft_x_ray_32', 'soft_x_ray_33',
 'soft_x_ray_34', 'soft_x_ray_35', 'soft_x_ray_36', 'soft_x_ray_37', 'soft_x_ray_38', 'poloidal_mirnov_probes_01', 'poloidal_mirnov_probes_02',
 'poloidal_mirnov_probes_03', 'poloidal_mirnov_probes_04', 'poloidal_mirnov_probes_05', 'poloidal_mirnov_probes_06',
 'poloidal_mirnov_probes_07', 'poloidal_mirnov_probes_08', 'poloidal_mirnov_probes_09', 'poloidal_mirnov_probes_10',
 'poloidal_mirnov_probes_11', 'poloidal_mirnov_probes_12', 'poloidal_mirnov_probes_13', 'poloidal_mirnov_probes_14',
 'poloidal_mirnov_probes_15', 'poloidal_mirnov_probes_16', 'poloidal_mirnov_probes_17', 'poloidal_mirnov_probes_18',
 'toroidal_mirnov_probes_01', 'toroidal_mirnov_probes_02', 'toroidal_mirnov_probes_03', 'toroidal_mirnov_probes_04',
 'toroidal_mirnov_probes_05', 'toroidal_mirnov_probes_06', 'toroidal_mirnov_probes_07', 'toroidal_mirnov_probes_09',
 'toroidal_mirnov_probes_10', 'density_measurement1', 'density_measurement2', 'density_measurement3', 'density_measurement4', 'greenwald_fraction',
 'lm_proxy', 'q95_proxy', 'ip_error_fraction', 'radiation_fraction', 'rotating_mode_proxy', 'unknown_bp_ka_top', 'unknown_bp_ef_top', 'unknown_bp_ka_bot', 'unknown_aminor', 'unknown_betan', 'unknown_kappa', 'unknown_li', 'unknown_wplasm',
 'programmed_plasma_current']
print(len(col_features))

113


<div style="font-weight: bold; font-size: 45px;" >
    Modelling
</div>

<div style="font-size: 30px;">
    Modelling using Xgboost and Catboost
</div>


In [55]:
import xgboost as xgb
import catboost as catb
import pandas as pd

import sklearn.preprocessing as preprocess
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
from tqdm.auto import tqdm
import pandas as pd
import joblib




<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [80]:
seed = 2020  # seed
fold_no = 12

skf = model_selection.StratifiedKFold(n_splits=fold_no, shuffle=True, random_state=seed)
scores = {'F1_SCORE': [], 'LOG_SCORE': [], 'recall': [], 'ACC': []}
preds_catb = []  # Store CatBoost predictions
preds_xgb = []   # Store XGBoost predictions
preds_combined = []  # Store combined predictions

ratio1 = 0.5
ratio2 = 0.5

X_, y_ = train_df[col_features].fillna(np.inf).replace(np.NAN, np.inf), train_df['Is_disrupt'].astype(int)
test_new = test_cmod_df[col_features].fillna(np.inf).replace(np.NAN, np.inf)

# creating a for loop for the kfold
i = 1
for train, test_ in skf.split(X_, y_):
    x_train, x_test, y_train, y_test = X_.iloc[train], X_.iloc[test_], y_.iloc[train], y_.iloc[test_]

    # CatBoost
    estimator1 = catb.CatBoostClassifier(iterations=40000, has_time=True, bootstrap_type='No', random_strength=0,
                                          learning_rate=0.011,
                                          use_best_model=True,
                                          reg_lambda=6, random_seed=39838,
                                          eval_metric='F1',
                                          early_stopping_rounds=1000, verbose=True)
    estimator1.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=5000)

    # XGBoost
    estimator2 = xgb.XGBClassifier(learning_rate=0.0023, n_estimators=1501, random_state=42, missing=np.inf,
                                   scale_pos_weight=1.2, max_depth=40, reg_lambda=0.8)
    estimator2.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=5000)

    # Save CatBoost model
    joblib.dump(estimator1, f'../tod/package1/catb{i}.pkl')

    # Save XGBoost model
    joblib.dump(estimator2, f'../tod/package1/xgb{i}.pkl')

    # Load models
    estimator1 = joblib.load(f'../tod/package1/catb{i}.pkl')
    estimator2 = joblib.load(f'../tod/package1/xgb{i}.pkl')

    # Predictions
    pred_catb = estimator1.predict_proba(test_new)[:, 1]
    pred_xgb = estimator2.predict_proba(test_new)[:, 1]

    print('Number of splits trained {} '.format(i))
    pred_combined = pred_catb * ratio1 + pred_xgb * ratio2

    # Validations
    valid_probs_catb = estimator1.predict_proba(x_test)[:, 1]
    valid_probs_xgb = estimator2.predict_proba(x_test)[:, 1]

    valid_preds_catb = [0 if i < 0.51 else 1 for i in valid_probs_catb]
    valid_preds_xgb = [0 if i < 0.51 else 1 for i in valid_probs_xgb]

    f1_score_catb = metrics.f1_score(y_test, valid_preds_catb, average='weighted')
    f1_score_xgb = metrics.f1_score(y_test, valid_preds_xgb, average='weighted')

    log_score_catb = metrics.log_loss(y_test, valid_probs_catb)
    log_score_xgb = metrics.log_loss(y_test, valid_probs_xgb)

    recallscore_catb = metrics.recall_score(y_test, valid_preds_catb)
    recallscore_xgb = metrics.recall_score(y_test, valid_preds_xgb)

    ACCscore_catb = metrics.accuracy_score(y_test, valid_preds_catb)
    ACCscore_xgb = metrics.accuracy_score(y_test, valid_preds_xgb)

    print(f'CatBoost - log_loss: {log_score_catb} F1_score: {f1_score_catb} acc_score: {ACCscore_catb} , recall: {recallscore_catb}')
    print(f'XGBoost - log_loss: {log_score_xgb} F1_score: {f1_score_xgb} acc_score: {ACCscore_xgb} , recall: {recallscore_xgb}')

    scores['LOG_SCORE'].append(log_score_catb)
    scores['F1_SCORE'].append(f1_score_catb)
    scores['recall'].append(recallscore_catb)
    scores['ACC'].append(ACCscore_catb)

    preds_catb.append(pred_catb)
    preds_xgb.append(pred_xgb)
    preds_combined.append(pred_combined)

    i += 1

# Save CatBoost predictions
final_df_catb = pd.DataFrame({'Shot_list': test_cmod_df.Shot_list.values, 'Is_disrupt': np.mean(preds_catb, axis=0)})
final_df_catb['Is_disrupt'] = [0 if i < 0.51 else 1 for i in final_df_catb['Is_disrupt'].values]
final_df_catb[['Shot_list', 'Is_disrupt']].to_csv('../tod/package1/sub_catb.csv', index=False)

# Save XGBoost predictions
final_df_xgb = pd.DataFrame({'Shot_list': test_cmod_df.Shot_list.values, 'Is_disrupt': np.mean(preds_xgb, axis=0)})
final_df_xgb['Is_disrupt'] = [0 if i < 0.51 else 1 for i in final_df_xgb['Is_disrupt'].values]
final_df_xgb[['Shot_list', 'Is_disrupt']].to_csv('../tod/package1/sub_xgb.csv', index=False)

# Save combined predictions
final_df_combined = pd.DataFrame({'Shot_list': test_cmod_df.Shot_list.values, 'Is_disrupt': np.mean(preds_combined, axis=0)})
final_df_combined['Is_disrupt'] = [0 if i < 0.51 else 1 for i in final_df_combined['Is_disrupt'].values]
final_df_combined[['Shot_list', 'Is_disrupt']].to_csv('../tod/package1/sub_combined.csv', index=False)


print('########################************ COMPLETED **********###################################################')


0:	learn: 0.8778626	test: 0.8602151	best: 0.8602151 (0)	total: 68.8ms	remaining: 45m 52s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9533678756
bestIteration = 2025

Shrink model to first 2026 iterations.
[0]	validation_0-logloss:0.66781
[1500]	validation_0-logloss:0.11469
Number of splits trained 1 
CatBoost - log_loss: 0.10203198338791639 F1_score: 0.9613520260071983 acc_score: 0.9616858237547893 , recall: 0.91
XGBoost - log_loss: 0.11468614888515809 F1_score: 0.9615299190686781 acc_score: 0.9616858237547893 , recall: 0.93
0:	learn: 0.8837429	test: 0.8526316	best: 0.8526316 (0)	total: 13.9ms	remaining: 9m 14s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9644670051
bestIteration = 1402

Shrink model to first 1403 iterations.
[0]	validation_0-logloss:0.66771
[1500]	validation_0-logloss:0.10676
Number of splits trained 2 
CatBoost - log_loss: 0.1055429071708014 F1_score: 0.9692239352549424 acc_score: 0.9693486590038314 , recall: 0.94
XG